In [1]:
from BODSDataExtractor.extractor import TimetableExtractor
import pandas as pd
import numpy as np
import requests
from lxml import etree as et
# from xml.etree import ElementTree as et
from time import sleep
from datetime import datetime, timedelta
import xml.dom.minidom
import xmltodict
import json
from bs4 import BeautifulSoup as bsoup
import time
import schedule
import os
import shutil
import pytz
import re

In [2]:
# api key
api_key = 'a3eda657579ef98d499ef515fbb5a32a86b22248'
fb_noc = ['FBRI']
org_name = 'First Bus'
timetable_dataset_id_list = [5815, 5814, 5813, 2283]
location_dataset_id_list = [5815, 5814, 5813, 2283]
bus_list = ['70', '72', '74', 'm1', 'm3', 'm4']
dataset_id = [2283, 2283, 5814, 5814]
# dataset_id = 699
latest_log = "Log initiating...\n"
output_dir = '../data/processed/timetable/'

In [3]:
def prettyprint(element, **kwargs):
    xml = et.tostring(element, pretty_print=True, **kwargs)
    print(xml.decode(), end='')

def df_insert_row(original_df, row_dictionary):
    # Convert row dictionary to DataFrame
    row_df = pd.DataFrame([row_dictionary])
    # Concatenate the original_df with the new row_df
    updated_df = pd.concat([original_df, row_df], ignore_index=True)
    return updated_df

# For lxml.etree only
def remove_namespace(response_content):
    tree = et.fromstring(response_content)
    # Remove annoying namespaces 
    for elem in tree.iter():
        # print(elem.tag)
        if '}' in elem.tag:
            elem.tag = elem.tag.split('}', 1)[1]  # Strip namespace
        # Remove namespace from attributes
        attribs = elem.attrib
        # print(elem.attrib)
        for attrib in list(attribs.keys()):
            if '}' in attrib:
                new_attrib = attrib.split('}', 1)[1]
                attribs[new_attrib] = attribs.pop(attrib)
    et.cleanup_namespaces(tree)
    return et.tostring(tree)

def timetable_data_search(bus_list, api_key, org_name='', catalog_location='./docs/location_data_catalogue.csv'):
    location_data_catalogue = pd.read_csv(catalog_location)
    location_data_catalogue = location_data_catalogue[location_data_catalogue['Organisation Name'].str.contains(org_name)]
    search_list = list(location_data_catalogue['Datafeed ID'])
    feed_id_list = []
    for feed_id in search_list:
        response = requests.get("https://data.bus-data.dft.gov.uk/api/v1/datafeed/"+str(feed_id)+"/?api_key="+api_key)
        bus_siri_file = et.fromstring(response.content)
        buses = bus_siri_file.findall('.//{http://www.siri.org.uk/siri}LineRef')
        unique_bus = sorted(list(set(str.lower(b.text) for b in buses)))
        for bus in bus_list:
            if unique_bus.count(str.lower(bus)) > 0:
                feed_id_list.append((bus, feed_id))
        sleep(0.1)
    return feed_id_list

def parse_runtime(duration):
    # Define the regular expression pattern
    pattern = re.compile(r'PT(?:(\d+)M)?(?:(\d+)S)?')
    match = pattern.match(duration)
    
    if not match:
        raise ValueError(f"Invalid duration format: {duration}")
    
    minutes = int(match.group(1)) if match.group(1) else 0
    seconds = int(match.group(2)) if match.group(2) else 0
    
    total_seconds = minutes * 60 + seconds
    return total_seconds

# Get the dataset ID

In [4]:
feed_id_list = []
pattern='|'.join(bus_list)
timetable_data_catalogue = pd.read_csv('../docs/timetables_data_catalogue.csv')
timetable_data_catalogue = timetable_data_catalogue[timetable_data_catalogue['Organisation Name'].str.contains(org_name, case=False, na=False)]
timetable_data_catalogue = timetable_data_catalogue[timetable_data_catalogue['XML:National Operator Code'].isin(fb_noc)]
timetable_data_catalogue = timetable_data_catalogue[timetable_data_catalogue['XML:Line Name'].str.contains(pattern, case=False, na=False)]
timetable_data_catalogue = timetable_data_catalogue[['XML:Line Name', 'Data set ID']].values.tolist()
timetable_data_catalogue

[['74 70', 2283.0],
 ['72', 2283.0],
 ['174 173', 5813.0],
 ['172', 5813.0],
 ['m4', 5814.0],
 ['m1', 5814.0]]

# Call the API

In [61]:
for id in dataset_id:
    timetable_raw = requests.get("https://data.bus-data.dft.gov.uk/api/v1/dataset/"+str(id)+"/?api_key="+api_key)
    response_text = json.loads(timetable_raw.content)
    print(response_text.get('url'))


https://data.bus-data.dft.gov.uk/timetable/dataset/2283/download/
https://data.bus-data.dft.gov.uk/timetable/dataset/2283/download/
https://data.bus-data.dft.gov.uk/timetable/dataset/5814/download/
https://data.bus-data.dft.gov.uk/timetable/dataset/5814/download/


# Process the Timetable

In [4]:
with open('../data/raw/timetable_70_74.xml', "r") as f:
        raw_xml = f.read()
timetable_data_xml = et.fromstring(remove_namespace(raw_xml))

Get all stops and their coordinates, put in a table

In [5]:
stop_info = pd.DataFrame()
for stop in timetable_data_xml.findall(".//StopPoints/AnnotatedStopPointRef"):
    # print('package start')
    tmp={}
    for data in stop.iter():
        if data.tag != 'AnnotatedStopPointRef':
            # print('an attribute')
            tmp[data.tag] = data.text
            # print(f'{data.tag}:{data.text}')
    stop_info = df_insert_row(stop_info, tmp)

Vehicle Journey

In [61]:
journey_info = pd.DataFrame()
for journey in timetable_data_xml.findall(".//VehicleJourneys/VehicleJourney"):
    # print('package start')
    tmp={}
    for data in journey.iter():
        if data.text != None:
            # print('an attribute')
            tmp[data.tag] = data.text
            # print(f'{data.tag}:{data.text}')
    if journey.find('StartDeadRun/PositioningLink/RunTime') != None:
        tmp['DeadRunRuntime'] = journey.find('StartDeadRun/PositioningLink/RunTime').text
    journey_info = df_insert_row(journey_info, tmp)
journey_info = journey_info[['LineRef','ServiceRef','JourneyPatternRef','BlockNumber','JourneyCode','VehicleJourneyCode','DepartureTime']].copy()

# This version omit the multiple temporal data provided by multiple VehicleJourneyTimingLink tags

Services

In [7]:
services_info = pd.DataFrame()
for service in timetable_data_xml.findall(".//Services/Service"):
    # print('package start')
    for line in service.findall("Lines/Line"):
        tmp={}
        tmp['LineId'] = line.attrib['id']
        tmp['LineName'] = line.find('LineName').text
        for out_attrib in line.find("OutboundDescription"):
            tmp['Direction'] = 'outbound'
            for attrib in out_attrib.iter():
                if attrib.text != None:
                    tmp[attrib.tag] = attrib.text
        services_info = df_insert_row(services_info, tmp)
        tmp={}
        tmp['LineId'] = line.attrib['id']
        tmp['LineName'] = line.find('LineName').text
        for in_attrib in line.find("InboundDescription"):
            tmp['Direction'] = 'inbound'
            for attrib in in_attrib.iter():
                if attrib.text != None:
                    tmp[attrib.tag] = attrib.text
        services_info = df_insert_row(services_info, tmp)

services_info = services_info[['LineId','LineName','Direction','Origin','Destination','Description']].copy()

In [8]:
std_services_info = pd.DataFrame()
for journey in timetable_data_xml.findall(".//Services/Service/StandardService/JourneyPattern"):
    # print('package start')
    tmp={}
    for attrib in journey.iter():
        tmp['JourneyPatternId'] = journey.attrib['id']
        if attrib.text != None:
            tmp[attrib.tag] = attrib.text
    std_services_info = df_insert_row(std_services_info, tmp)

std_services_info = std_services_info[['JourneyPatternId','DestinationDisplay','Direction','RouteRef','JourneyPatternSectionRefs']].copy()

In [9]:
service_lookup_table = (journey_info
                        .merge(std_services_info, 'left', left_on=['JourneyPatternRef'], right_on=['JourneyPatternId'])
                        .merge(services_info, 'left', left_on=['LineRef', 'Direction'], right_on=['LineId', 'Direction']))
service_lookup_table = service_lookup_table[['LineId','ServiceRef','LineName','Origin','Destination','Description','Direction',
                                             'JourneyPatternRef','BlockNumber','JourneyCode','VehicleJourneyCode','DepartureTime',
                                             'JourneyPatternId','DestinationDisplay','RouteRef','JourneyPatternSectionRefs']].copy()

Route Data

In [74]:
route_data = pd.DataFrame()
for route in timetable_data_xml.findall(".//Routes/Route"):
    # print(stop.attrib)
    tmp={}
    tmp['RouteId'] = route.attrib['id']
    for data in route.iter():
        if data.text != None:
            # print('an attribute')
            tmp[data.tag] = data.text
            # print(f'{data.tag}:{data.text}')
    route_data = df_insert_row(route_data, tmp)
route_data = route_data[['RouteId','PrivateCode','Description','RouteSectionRef']].copy()

In [11]:
tracking_data = pd.DataFrame()
for route in timetable_data_xml.findall(".//RouteSections/RouteSection"):
    # print(stop.attrib)
    tmp={}
    # tmp['RouteSelectionRef'] = route.attrib['id']
    for link in route.findall("RouteLink"):
        tmp['RouteSelectionRef'] = route.attrib['id']
        tmp['RouteLinkRef'] = link.attrib['id']
        tmp['StartPointRef'] = link.find("From/StopPointRef").text
        tmp['EndPointRef'] = link.find("To/StopPointRef").text
        tmp['Distance'] = link.find("Distance").text
        tmp['TrackingPoints'] = []
        for point in link.findall("Track/Mapping/Location"):
            tmp['TrackingPoints'].append([float(point.find("Latitude").text), float(point.find("Longitude").text)])
        tracking_data = df_insert_row(tracking_data, tmp)

In [89]:
timing_mapping_data = pd.DataFrame()
for journey_pattern_selection in timetable_data_xml.findall(".//JourneyPatternSections/JourneyPatternSection"):
    for timinglink in journey_pattern_selection.findall('JourneyPatternTimingLink'):
        tmp={}
        tmp['JourneyPatternSectionId'] = journey_pattern_selection.attrib['id']
        tmp['JourneyPatternTimingLinkId'] = timinglink.attrib['id']
        tmp['RouteLinkRef'] = timinglink.find('RouteLinkRef').text
        timing_mapping_data = df_insert_row(timing_mapping_data, tmp)

timing_data = pd.DataFrame()
for leg_runtime in timetable_data_xml.findall(".//VehicleJourneys/VehicleJourney"):
    # print(leg_runtime.find('VehicleJourneyCode').text)
    for timinglink in leg_runtime.findall('VehicleJourneyTimingLink'):
        tmp={}
        # print(timinglink.attrib['id'])
        tmp['VehicleJourneyTimingLinkId'] = timinglink.attrib['id']
        tmp['JourneyPatternTimingLinkRef'] = timinglink.find('JourneyPatternTimingLinkRef').text
        tmp['RunTime'] = parse_runtime(timinglink.find('RunTime').text)
        if timinglink.find('.//WaitTime') != None:
            tmp['WaitTime'] = parse_runtime(timinglink.find('.//WaitTime').text)
        if leg_runtime.find('.//BlockNumber') != None:
            tmp['BlockNumber'] = leg_runtime.find('.//BlockNumber').text
        timing_data = df_insert_row(timing_data, tmp)

# timing_data = timing_data.merge(timing_mapping_data, 'left', left_on=['JourneyPatternTimingLinkRef'], right_on=['JourneyPatternTimingLinkId']).copy()
# timing_data = timing_data[['VehicleJourneyTimingLinkId','JourneyPatternTimingLinkRef','RouteLinkRef','RunTime']]

# Extract Model Trips As GeoJSON

In [4]:
file_list = ['../data/raw/timetable_m1.xml', '../data/raw/timetable_m3.xml', '../data/raw/timetable_m4.xml', '../data/raw/timetable_70_74.xml', '../data/raw/timetable_72.xml']

In [10]:
master_service_lookup_table = pd.DataFrame()
master_tracking_data = pd.DataFrame()
master_timing_data = pd.DataFrame()
master_stop_info = pd.DataFrame()

for file in file_list:
    print(f'Processing {file}')
    with open(file, "r") as f:
            raw_xml = f.read()
    timetable_data_xml = et.fromstring(remove_namespace(raw_xml))

    print('\tGATHERING STOP INFORMATION')
    # Get stop codes
    stop_info = pd.DataFrame()
    for stop in timetable_data_xml.findall(".//StopPoints/AnnotatedStopPointRef"):
        # print('package start')
        tmp={}
        for data in stop.iter():
            if data.tag != 'AnnotatedStopPointRef':
                # print('an attribute')
                tmp[data.tag] = data.text
                # print(f'{data.tag}:{data.text}')
        stop_info = df_insert_row(stop_info, tmp)

    print('\tEXTRACTING JOURNEY INFORMATION')
    journey_info = pd.DataFrame()
    for journey in timetable_data_xml.findall(".//VehicleJourneys/VehicleJourney"):
        # print('package start')
        tmp={}
        for data in journey.iter():
            if data.text != None:
                # print('an attribute')
                tmp[data.tag] = data.text
                # print(f'{data.tag}:{data.text}')
        if journey.find('StartDeadRun/PositioningLink/RunTime') != None:
            tmp['DeadRunRuntime'] = journey.find('StartDeadRun/PositioningLink/RunTime').text
        journey_info = df_insert_row(journey_info, tmp)
    journey_info = journey_info[journey_info.columns.intersection(['LineRef','ServiceRef','JourneyPatternRef','BlockNumber','JourneyCode','VehicleJourneyCode','DepartureTime'])].copy()
    # This version omit the multiple temporal data provided by multiple VehicleJourneyTimingLink tags

    print('\tEXTRACTING SERVICE INFORMATION')
    services_info = pd.DataFrame()
    for service in timetable_data_xml.findall(".//Services/Service"):
        # print('package start')
        for line in service.findall("Lines/Line"):
            tmp={}
            tmp['LineId'] = line.attrib['id']
            tmp['LineName'] = line.find('LineName').text
            for out_attrib in line.find("OutboundDescription"):
                tmp['Direction'] = 'outbound'
                for attrib in out_attrib.iter():
                    if attrib.text != None:
                        tmp[attrib.tag] = attrib.text
            services_info = df_insert_row(services_info, tmp)
            tmp={}
            tmp['LineId'] = line.attrib['id']
            tmp['LineName'] = line.find('LineName').text
            for in_attrib in line.find("InboundDescription"):
                tmp['Direction'] = 'inbound'
                for attrib in in_attrib.iter():
                    if attrib.text != None:
                        tmp[attrib.tag] = attrib.text
            services_info = df_insert_row(services_info, tmp)
    services_info = services_info[services_info.columns.intersection(['LineId','LineName','Direction','Origin','Destination','Description'])].copy()

    std_services_info = pd.DataFrame()
    for journey in timetable_data_xml.findall(".//Services/Service/StandardService/JourneyPattern"):
        # print('package start')
        tmp={}
        for attrib in journey.iter():
            tmp['JourneyPatternId'] = journey.attrib['id']
            if attrib.text != None:
                tmp[attrib.tag] = attrib.text
        std_services_info = df_insert_row(std_services_info, tmp)
    std_services_info = std_services_info[std_services_info.columns.intersection(['JourneyPatternId','DestinationDisplay','Direction','RouteRef','JourneyPatternSectionRefs'])].copy()

    print('\tMERGING SERVICE INFORMATION')
    service_lookup_table = (journey_info
                            .merge(std_services_info, 'left', left_on=['JourneyPatternRef'], right_on=['JourneyPatternId'])
                            .merge(services_info, 'left', left_on=['LineRef', 'Direction'], right_on=['LineId', 'Direction']))
    service_lookup_table['JourneyCode'] = service_lookup_table['JourneyCode'].astype(int).astype(str).str.zfill(4)
    service_lookup_table = service_lookup_table[service_lookup_table.columns.intersection(['LineId','ServiceRef','LineName','Origin','Destination','Description','Direction',
                                                'JourneyPatternRef','BlockNumber','JourneyCode','DepartureTime',
                                                'JourneyPatternId','DestinationDisplay','RouteRef','JourneyPatternSectionRefs'])].copy()
    
    all_service_list = services_info['LineName'].unique().tolist()

    print('\tEXTRACTING SPATIAL INFORMATION')
    route_data = pd.DataFrame()
    for route in timetable_data_xml.findall(".//Routes/Route"):
        # print(stop.attrib)
        tmp={}
        tmp['RouteId'] = route.attrib['id']
        for data in route.iter():
            if data.text != None:
                # print('an attribute')
                tmp[data.tag] = data.text
                # print(f'{data.tag}:{data.text}')
        route_data = df_insert_row(route_data, tmp)
    route_data = route_data[route_data.columns.intersection(['RouteId','PrivateCode','Description','RouteSectionRef'])].copy()

    tracking_data = pd.DataFrame()
    for route in timetable_data_xml.findall(".//RouteSections/RouteSection"):
        # print(stop.attrib)
        tmp={}
        # tmp['RouteSelectionRef'] = route.attrib['id']
        for link in route.findall("RouteLink"):
            tmp['RouteSectionRef'] = route.attrib['id']
            tmp['RouteLinkRef'] = link.attrib['id']
            tmp['StartPointRef'] = link.find("From/StopPointRef").text
            tmp['EndPointRef'] = link.find("To/StopPointRef").text
            tmp['Distance'] = float(link.find("Distance").text)
            tmp['TrackingPoints'] = []
            for point in link.findall("Track/Mapping/Location"):
                tmp['TrackingPoints'].append([float(point.find("Longitude").text), float(point.find("Latitude").text)])
            tracking_data = df_insert_row(tracking_data, tmp)

    tracking_data = tracking_data.merge(route_data[['RouteId', 'RouteSectionRef']], 'left', left_on=['RouteSectionRef'], right_on=['RouteSectionRef']).copy()
    tracking_data = tracking_data[tracking_data.columns.intersection(['RouteId','RouteSectionRef','RouteLinkRef','StartPointRef','EndPointRef','Distance','TrackingPoints'])].copy()

    service_count = len(all_service_list)
    tracker_count = len(tracking_data)
    tracking_data = pd.concat([tracking_data]*service_count, ignore_index=True)
    tracking_data = pd.concat([tracking_data, pd.DataFrame({'LineRef':[line for line in all_service_list for _ in range(tracker_count)]})], axis=1)

    print('\tEXTRACTING TEMPORAL INFORMATION')
    timing_mapping_data = pd.DataFrame()
    for journey_pattern_selection in timetable_data_xml.findall(".//JourneyPatternSections/JourneyPatternSection"):
        for timinglink in journey_pattern_selection.findall('JourneyPatternTimingLink'):
            tmp={}
            tmp['JourneyPatternSectionId'] = journey_pattern_selection.attrib['id']
            tmp['JourneyPatternTimingLinkId'] = timinglink.attrib['id']
            tmp['RouteLinkRef'] = timinglink.find('RouteLinkRef').text
            timing_mapping_data = df_insert_row(timing_mapping_data, tmp)

    timing_data = pd.DataFrame()
    for leg_runtime in timetable_data_xml.findall(".//VehicleJourneys/VehicleJourney"):
        # print(leg_runtime.find('VehicleJourneyCode').text)
        for timinglink in leg_runtime.findall('VehicleJourneyTimingLink'):
            tmp={}
            # print(timinglink.attrib['id'])
            tmp['VehicleJourneyTimingLinkId'] = timinglink.attrib['id']
            tmp['JourneyPatternTimingLinkRef'] = timinglink.find('JourneyPatternTimingLinkRef').text
            tmp['RunTime'] = parse_runtime(timinglink.find('RunTime').text)
            if timinglink.find('.//WaitTime') != None:
                tmp['WaitTime'] = parse_runtime(timinglink.find('.//WaitTime').text)
            if leg_runtime.find('.//BlockNumber') != None:
                tmp['BlockNumber'] = leg_runtime.find('.//BlockNumber').text
            timing_data = df_insert_row(timing_data, tmp)

    timing_data = timing_data.merge(timing_mapping_data, 'left', left_on=['JourneyPatternTimingLinkRef'], right_on=['JourneyPatternTimingLinkId']).copy()
    timing_data = timing_data[timing_data.columns.intersection(['JourneyPatternSectionId','BlockNumber','JourneyPatternTimingLinkRef','RouteLinkRef','RunTime', 'WaitTime'])]

    file_line_name = '_'.join(services_info['LineName'].unique())
    print(f'\tSAVE RESULTS FOR {file_line_name}')
    
    master_service_lookup_table = pd.concat([master_service_lookup_table, service_lookup_table], ignore_index=True)
    master_tracking_data = pd.concat([master_tracking_data, tracking_data], ignore_index=True)
    master_timing_data = pd.concat([master_timing_data, timing_data], ignore_index=True)
    master_stop_info = pd.concat([master_stop_info, stop_info], ignore_index=True)
        
    
    # service_lookup_table.to_csv(output_dir+'service_lookup_table_'+str(file_line_name)+'.csv',index=False)
    # tracking_data.to_csv(output_dir+'tracking_data'+str(file_line_name)+'.csv',index=False)
    # timing_data.to_csv(output_dir+'timing_data_'+str(file_line_name)+'.csv',index=False)
    # stop_info.to_csv(output_dir+'stop_info_'+str(file_line_name)+'.csv',index=False)

master_service_lookup_table.drop_duplicates().to_pickle(output_dir+'master_service_lookup_table.pkl')
master_tracking_data.to_pickle(output_dir+'master_tracking_data.pkl')
master_timing_data.drop_duplicates().to_pickle(output_dir+'master_timing_data.pkl')
master_stop_info.drop_duplicates().to_pickle(output_dir+'master_stop_info.pkl')

Processing ../data/raw/timetable_m1.xml
	GATHERING STOP INFORMATION
	EXTRACTING JOURNEY INFORMATION
	EXTRACTING SERVICE INFORMATION
	MERGING SERVICE INFORMATION
	EXTRACTING SPATIAL INFORMATION
	EXTRACTING TEMPORAL INFORMATION
	SAVE RESULTS FOR m1
Processing ../data/raw/timetable_m3.xml
	GATHERING STOP INFORMATION
	EXTRACTING JOURNEY INFORMATION
	EXTRACTING SERVICE INFORMATION
	MERGING SERVICE INFORMATION
	EXTRACTING SPATIAL INFORMATION
	EXTRACTING TEMPORAL INFORMATION
	SAVE RESULTS FOR m3_m3x
Processing ../data/raw/timetable_m4.xml
	GATHERING STOP INFORMATION
	EXTRACTING JOURNEY INFORMATION
	EXTRACTING SERVICE INFORMATION
	MERGING SERVICE INFORMATION
	EXTRACTING SPATIAL INFORMATION
	EXTRACTING TEMPORAL INFORMATION
	SAVE RESULTS FOR m4
Processing ../data/raw/timetable_70_74.xml
	GATHERING STOP INFORMATION
	EXTRACTING JOURNEY INFORMATION
	EXTRACTING SERVICE INFORMATION
	MERGING SERVICE INFORMATION
	EXTRACTING SPATIAL INFORMATION
	EXTRACTING TEMPORAL INFORMATION
	SAVE RESULTS FOR 74_70
Pr